# 환경 설정

In [1]:
pip install -q langchain langchain-openai google-search-results

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install langchain-community

In [2]:
import os

os.environ['OPENAI_API_KEY'] = 'api_key'
os.environ['SERPAPI_API_KEY'] = 'api_key'

# News 데이터 수집 - SerpAPI 활용
- 인증키 발급 필요 (월 100회 무료)
- https://serpapi.com/

In [ ]:
from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "google_news",
    "gl": "KR",
    "hl": "ko",
}
search = SerpAPIWrapper(params=params)

search.run("클로드")

In [11]:
results = search.run("클로드")
len(results)

97

In [12]:
results[0]

{'position': 1,
 'title': "앤트로픽, '클로드 3.5 소네트' 출시...'GPT-4o' 제치고 최강 모델 탈환",
 'source': {'name': 'AI타임스',
  'icon': 'https://encrypted-tbn0.gstatic.com/faviconV2?url=https://www.aitimes.com&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
 'link': 'https://www.aitimes.com/news/articleView.html?idxno=160843',
 'thumbnail': 'https://cdn.aitimes.com/news/thumbnail/202406/160843_173716_83_v150.jpg',
 'date': '06/21/2024, 07:00 AM, +0000 UTC'}

In [7]:
results[0].keys()

dict_keys(['position', 'title', 'source', 'link', 'thumbnail', 'date'])

In [13]:
url = results[0]['link']
url

'https://www.aitimes.com/news/articleView.html?idxno=160843'

### LangChain WebBaseLoader 활용

In [ ]:
pip install beautifulsoup4

In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup

loader = WebBaseLoader(url)
docs = loader.load()

docs[0]

In [20]:
# 뉴스기사에서 본문 텍스트를 추출하는 함수
def extract_longest_text(text):
    # 개행문자(\n\n)를 기준으로 텍스트를 분할
    segments = text.split('\n')
    # 가장 긴 텍스트 조각을 찾음
    longest_segment = max(segments, key=len)
    return longest_segment

text = docs[0].page_content

longest_text = extract_longest_text(text)
print("가장 긴 텍스트:\n", longest_text)

가장 긴 텍스트:
 대학원 수준의 추론 능력(GPQA)는 59.4%로 GPT-4o(53.6%)를 크게 앞섰고, 코딩 능력(HumanEval) 역시 92.0%로 GPT-4o( 90.2%) 대비 1.8%포인트 높았다. 또 클로드 3.5 소네트는 중간 크기의 모델이지만, 전 세대 최고 모델인 클로드 3 오퍼스의 성능을 모두 앞질렀다. 


In [ ]:
# SerpAPI 검색 결과를 순회하면서 본문 텍스트를 추출하여 기존 딕셔너리에 추가

from tqdm import tqdm
import time

new_results = []
for result in tqdm(results):
    try:
        url = result['link']
        loader = WebBaseLoader(url)
        docs = loader.load()
        text = docs[0].page_content
        longest_text = extract_longest_text(text)
        result['content'] = longest_text
        new_results.append(result)
    except:
        pass


# 결과 확인
print("새로운 결과 개수:", len(new_results))
new_results[0]

# 판다스 데이터프레임으로 변환

In [ ]:
pip install pandas

In [24]:
import pandas as pd
data = pd.DataFrame(new_results)
data.head()

,position,title,source,link,thumbnail,date,content
0,1,"앤트로픽, '클로드 3.5 소네트' 출시...'GPT-4o' 제치고 최강 모델 탈환","{'name': 'AI타임스', 'icon': 'https://encrypted-t...",https://www.aitimes.com/news/articleView.html?...,https://cdn.aitimes.com/news/thumbnail/202406/...,"06/21/2024, 07:00 AM, +0000 UTC",대학원 수준의 추론 능력(GPQA)는 59.4%로 GPT-4o(53.6%)를 크게 ...
1,2,"GPT-4o도 제쳤다… 앤트로픽, 클로드 3.5 소네트 출시 ‘경쟁 가열’","{'name': '더밀크 The Miilk', 'icon': 'https://lh3...",https://www.themiilk.com/articles/a9e7155d0,https://dsi523du1o5iq.cloudfront.net/fit-in/90...,"06/20/2024, 07:00 AM, +0000 UTC","다리오 아모데이는 컴퓨터 공학, 인공 신경망 등 AI 관련 학문을 전문적으로 연구한..."
2,3,"앤트로픽, 안드로이드용 '클로드 앱' 무료 출시... AI 챗봇 시장 경쟁 가속","{'name': '인공지능신문', 'icon': 'https://lh3.google...",https://www.aitimes.kr/news/articleView.html?i...,https://cdn.aitimes.kr/news/photo/202407/31689...,"07/17/2024, 07:00 AM, +0000 UTC",새로운 클로드 안드로이드 앱은 가장 강력한 모델인 클로드 3.5 소네트(Claude...
3,4,앤트로픽 '클로드' 안드로이드용 앱 출시…실시간 번역 OK,"{'name': '디지털투데이', 'icon': 'https://encrypted-...",https://www.digitaltoday.co.kr/news/articleVie...,https://cdn.digitaltoday.co.kr/news/photo/2024...,"07/17/2024, 07:00 AM, +0000 UTC","이는 앞서 5월에 출시된 iOS 앱 버전과 동일하게 작동하며, 클로드 3.5 소네트..."
4,5,"‘제미니’와 ‘클로드’, 어느 것이 낫나?","{'name': '애플경제', 'icon': 'https://lh3.googleus...",https://www.apple-economy.com/news/articleView...,https://cdn.apple-economy.com/news/photo/20240...,"07/08/2024, 07:00 AM, +0000 UTC","제미니와 클로드는 모두 아이디어나 아이템을 제안하는데도 능숙하다. 배우고 싶은 것,..."


# 뉴스 요약

In [25]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Write a concise summary of the following in Korean Hangul (한글):
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'대학원 수준의 추론 능력(GPQA)은 59.4%로 GPT-4o를 크게 앞섰고, 코딩 능력(HumanEval)도 92.0%로 GPT-4o보다 높았다. 클로드 3.5 소네트는 중간 크기 모델임에도 클로드 3 오퍼스의 성능을 초월했다.'

In [50]:
# 뉴스 본문을 입력으로 사용하여 요약 결과를 생성하는 함수
def summarize_news(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 요약
df_test = data.head(3)
df_test['summary'] = df_test['content'].apply(summarize_news)

df_test

/var/folders/nd/kfl3k2197fv3vr63kpjrnrg00000gn/T/ipykernel_5239/3536674286.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['summary'] = df_test['content'].apply(summarize_news)


,position,title,source,link,thumbnail,date,content,summary
0,1,이차전지 산업 성과와 향후 과제는,"{'name': '전자신문', 'icon': 'https://encrypted-tb...",https://www.etnews.com/20240311000263,https://news.google.com/api/attachments/CC8iL0...,"03/11/2024, 05:26 AM, +0000 UTC",민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 11일 서울 강남구 한국...,"한국에서 민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 열렸고, 안덕근..."
1,2,"윤신애 군산시의원, '군산새만금 이차전지산업 발전위원회 설치 및 운영조례안' 상임위...","{'name': '아시아뉴스전북', 'icon': 'https://lh3.googl...",http://www.mjeonbuk.com/news/articleView.html?...,https://news.google.com/api/attachments/CC8iK0...,"03/12/2024, 06:56 AM, +0000 UTC","윤신애 의원은 “작년「군산새만금신항 발전위원회 설치 및 운영조례」를 발의, 발전위원...",윤신애 의원은 작년 발의한 군산 새만금신항 발전위원회 설치 및 운영조례로 새만금 개...
2,3,"[이차전지특구-충북②] 충북도 김명규 경제부지사, “국내 1위 넘어 글로벌 배터리 ...","{'name': '인더스트리뉴스', 'icon': 'https://lh3.googl...",https://www.industrynews.co.kr/news/articleVie...,https://news.google.com/api/attachments/CC8iK0...,"03/07/2024, 01:00 AM, +0000 UTC",인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...


In [51]:
df_test[['content', 'summary']]

,content,summary
0,민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 11일 서울 강남구 한국...,"한국에서 민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 열렸고, 안덕근..."
1,"윤신애 의원은 “작년「군산새만금신항 발전위원회 설치 및 운영조례」를 발의, 발전위원...",윤신애 의원은 작년 발의한 군산 새만금신항 발전위원회 설치 및 운영조례로 새만금 개...
2,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...


# 키워드 추출

In [52]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Please extract 3 key words from the following content in Korean Hangul (한글) and separate them with commas (,):
"{text}"
Key words:"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'합동, 배터리, 현안'

In [53]:
# 뉴스 본문을 입력으로 사용하여 핵심 키워드를 추출하는 함수
def extract_keywords(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 추출
df_test['keywords'] = df_test['content'].apply(extract_keywords)

df_test[['content', 'summary', 'keywords']]

/var/folders/nd/kfl3k2197fv3vr63kpjrnrg00000gn/T/ipykernel_5239/77447675.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords'] = df_test['content'].apply(extract_keywords)


,content,summary,keywords
0,민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 11일 서울 강남구 한국...,"한국에서 민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 열렸고, 안덕근...","배터리, 수출, 현안"
1,"윤신애 의원은 “작년「군산새만금신항 발전위원회 설치 및 운영조례」를 발의, 발전위원...",윤신애 의원은 작년 발의한 군산 새만금신항 발전위원회 설치 및 운영조례로 새만금 개...,"윤신애, 새만금, 발전위원회"
2,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,"이차전지, 국내, 충북"


# 뉴스 카테고리 분류

In [54]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Based on the following content, please classify the news into the appropriate category and provide the category name in Korean Hangul (한글):
"{text}"
News Category:"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

'경제 (Economy) - 경제'

In [55]:
# 뉴스 본문을 입력으로 사용하여 카테고리를 분류하는 함수
def classify_news_category(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 카테고리 분류
df_test['category'] = df_test['content'].apply(classify_news_category)

df_test[['content', 'summary', 'keywords', 'category']]

/var/folders/nd/kfl3k2197fv3vr63kpjrnrg00000gn/T/ipykernel_5239/2226806532.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['category'] = df_test['content'].apply(classify_news_category)


,content,summary,keywords,category
0,민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 11일 서울 강남구 한국...,"한국에서 민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 열렸고, 안덕근...","배터리, 수출, 현안",경제 (Gyeongje)
1,"윤신애 의원은 “작년「군산새만금신항 발전위원회 설치 및 운영조례」를 발의, 발전위원...",윤신애 의원은 작년 발의한 군산 새만금신항 발전위원회 설치 및 운영조례로 새만금 개...,"윤신애, 새만금, 발전위원회",정치
2,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,"이차전지, 국내, 충북",산업 (Industry)


### 키워드에서 한글만 추출

In [57]:
import re
korean_word = re.findall(r'[가-힣]+', df_test['category'][0])[0]
korean_word

'경제'

In [58]:
# 람다 함수를 사용하여 카테고리에서 한글 단어만 추출 
df_test['category'].apply(lambda x: re.findall(r'[가-힣]+', x)[0])

0    경제
1    정치
2    산업
Name: category, dtype: object

In [59]:
df_test['category'] = df_test['category'].apply(lambda x: re.findall(r'[가-힣]+', x)[0])
df_test[['content', 'summary', 'keywords', 'category']]

/var/folders/nd/kfl3k2197fv3vr63kpjrnrg00000gn/T/ipykernel_5239/1981466824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['category'] = df_test['category'].apply(lambda x: re.findall(r'[가-힣]+', x)[0])


,content,summary,keywords,category
0,민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 11일 서울 강남구 한국...,"한국에서 민·관 합동 배터리 얼라이언스 제3차 수출·현안 전략회의가 열렸고, 안덕근...","배터리, 수출, 현안",경제
1,"윤신애 의원은 “작년「군산새만금신항 발전위원회 설치 및 운영조례」를 발의, 발전위원...",윤신애 의원은 작년 발의한 군산 새만금신항 발전위원회 설치 및 운영조례로 새만금 개...,"윤신애, 새만금, 발전위원회",정치
2,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,인더스트리뉴스는 2024년 배터리리포트 기획으로 ‘국가첨단전략산업 이차전지 특화단지...,"이차전지, 국내, 충북",산업
